In [ ]:
%pylab inline

In [ ]:
from numpy import array
import openturns as ot
from openturns.viewer import View
# set seed for random number generator to assure the reproducibility of the analysis
ot.RandomGenerator.SetSeed(1)
# set command to call R from within the python script
ot.ResourceMap.Set('R-executable-command', 'R')

# Step A - Model definition

In [ ]:
dim = 4
model = ot.SymbolicFunction(["E", "F", "L", "I"], ["F*L^3/(3*E*I)"])

# Step B - Modelling of the input uncertainty

We first define the marginal distributions

In [ ]:
# Young's modulus E: data-based kernel distribution
kernel = ot.KernelSmoothing()
sample = ot.Sample.ImportFromCSVFile("sample_E.csv")  # in Pa
dist_E = kernel.build(sample)

# Load F: lognormal
F_para = ot.LogNormalMuSigma(3e4, 9e3, 15e3) # in N
dist_F = ot.ParametrizedDistribution(F_para)

# Length L: uniform
dist_L = ot.Uniform(250, 260) # in cm

# Moment of inertia I: beta
dist_I = ot.Beta(2.5, 4, 310., 450.) # in cm^4
#dist_E.computeQuantile(0.5)

Draw the CDF of the parameters, e.g. the Young's modulus E

In [ ]:
graph = dist_E.drawCDF()
graph.setLegends(['E'])
graph.setTitle('')
graph.setXTitle('E')
View(graph).show()

In [ ]:
graph = dist_F.drawCDF()
graph.setLegends(['F'])
graph.setTitle('')
graph.setXTitle('F')
View(graph).show()

Then we specify the copula: a Gaussian copula is assumed

In [ ]:
R = ot.CorrelationMatrix(dim)
R[2,3] = -.2 # specify the correlation between variables IFF necessary, otherwise the standard correlation matrix is a unit matrix
myCopula = ot.NormalCopula(ot.NormalCopula.GetCorrelationFromSpearmanCorrelation(R)) # be sure to introduce the Pearson correlation matrix

Lastly we gather the marginals and the copula to create the joint distribution of the inputs

In [ ]:
myDistribution = ot.ComposedDistribution([dist_E, dist_F, dist_L, dist_I], myCopula)
# generate a set of values of the random parameters
X = myDistribution.getSample(10)

# Step C - Uncertainty propagation based on polynomial chaos

## Construct the polynomial chaos approximation

The types of 1D polynomial families are defined

In [ ]:
polyColl = ot.PolynomialFamilyCollection(dim)
polyColl[0] = ot.StandardDistributionPolynomialFactory(dist_E)
polyColl[1] = ot.LaguerreFactory() # beware of ad-hoc polynomials with the lognormal distribution
polyColl[2] = ot.StandardDistributionPolynomialFactory(dist_L)
polyColl[3] = ot.StandardDistributionPolynomialFactory(dist_I)

Define the strategy for enumerating the elements of the multivariate tensorized basis

In [ ]:
enumerateFunction = ot.LinearEnumerateFunction(dim)

Determine the number of elements to be retained in the chaos series

In [ ]:
p = 7 # total degree, integer
P = enumerateFunction.getStrataCumulatedCardinal(p) # number of chaos terms

Define the multivariate basis

In [ ]:
multivariateBasis = ot.OrthogonalProductPolynomialFactory(polyColl, enumerateFunction)

Specify the strategy for computing the chaos coefficients: sparse least squares (based on Least Angle Regression)

In [ ]:
basisSequenceFactory = ot.LARS()
fittingAlgorithm = ot.CorrectedLeaveOneOut()
approximationAlgorithm = ot.LeastSquaresMetaModelSelectionFactory(basisSequenceFactory, fittingAlgorithm)
N = 500 # size of the experimental design
evalStrategy = ot.LeastSquaresStrategy(ot.MonteCarloExperiment(N), approximationAlgorithm) # Monte Carlo design
truncatureBasisStrategy = ot.FixedStrategy(multivariateBasis, P)

Construct the chaos metamodel

In [ ]:
polynomialChaosAlgorithm = ot.FunctionalChaosAlgorithm(model, myDistribution, truncatureBasisStrategy, evalStrategy)
polynomialChaosAlgorithm.run()

Get the result

In [ ]:
result = polynomialChaosAlgorithm.getResult()
metamodel = result.getMetaModel()
chaosRV = ot.FunctionalChaosRandomVector(result)

## Validate the metamodel

Generate a validation sample (different from the training sample)

In [ ]:
n_valid = 1000
inputSample = myDistribution.getSample(n_valid)
outputSample = model(inputSample)

Get the corresponding chaos-based predictions

In [ ]:
outputChaosSample = metamodel(inputSample)

Compute the relative mean-square accuracy (so-called Q2 coefficient)

In [ ]:
residuals = outputSample - outputChaosSample
Q2 = 1. - residuals.computeVariance()[0] / outputSample.computeVariance()[0]
print("Q2 coefficient (relative accuracy): %.4f" % Q2)

Plot the actual vs the predicted outputs

In [ ]:
val = ot.MetaModelValidation(inputSample, outputSample, metamodel)
graph = val.drawValidation()
View(graph).show()

## Post-processing of the chaos expansion

### Computation of the output mean and standard deviation from the chaos coefficients

In [ ]:
print("Mean: %f" % chaosRV.getMean()[0])
print("Standard deviation: %f" % sqrt(chaosRV.getCovariance()[0,0]))

Draw the empirical PDF of the response obtained with Kernel density estimation technique

In [ ]:
inputBigSample = myDistribution.getSample(10000)
outputBigsample = metamodel(inputBigSample)
kernel2 = ot.KernelSmoothing()
fittedDist = kernel2.build(outputBigsample)
graph = fittedDist.drawPDF()
View(graph).show()

### Computation of Sobol' sensitivity indices

In [ ]:
chaosSI = ot.FunctionalChaosSobolIndices(result)

# Main-effects
for i, Xi in enumerate(["E", "F", "L", "I"]):
    sobolIndex = chaosSI.getSobolIndex(i)
    print("Sobol' sensitivity index to "+Xi+":    %f" %sobolIndex)
    
# Total indices
for i, Xi in enumerate(["E", "F", "L", "I"]):
    sobolTotalIndex = chaosSI.getSobolTotalIndex(i)
    print("Total Sobol' sensitivity index to "+Xi+":    %f" %sobolTotalIndex)

In [ ]:
# interaction effects between first and second variables
chaosSI.getSobolTotalIndex([0,1])

### Reliability analysis using polynomial chaos and MCS

In [ ]:
# Reliability analysis using a (large) Monte Carlo sample of the chaos approximation
#
threshold = 30
myEvent = ot.ThresholdEvent(chaosRV, ot.ComparisonOperator(ot.Greater()), threshold)
myEvent.setName("Deviation > 30 cm")
#
maximumOuterSampling = 40000
blockSize = 100
coefficientOfVariation = 0.10
#
experiment = ot.MonteCarloExperiment()
myAlgoMonteCarlo = ot.ProbabilitySimulationAlgorithm(myEvent, experiment)
myAlgoMonteCarlo.setMaximumOuterSampling(maximumOuterSampling)
myAlgoMonteCarlo.setBlockSize(blockSize)
myAlgoMonteCarlo.setMaximumCoefficientOfVariation(coefficientOfVariation)
myAlgoMonteCarlo.setConvergenceStrategy(ot.HistoryStrategy(ot.Full()))
#
myAlgoMonteCarlo.run()

In [ ]:
print( "Number of evaluations of the limit state function = ", myAlgoMonteCarlo.getResult().getOuterSampling()* myAlgoMonteCarlo.getResult().getBlockSize() )
print( "Monte Carlo probability estimation = ", myAlgoMonteCarlo.getResult().getProbabilityEstimate() )
print( "Variance of the Monte Carlo probability estimator = ", myAlgoMonteCarlo.getResult().getVarianceEstimate() )
print( "0.95 Confidence Interval = [", myAlgoMonteCarlo.getResult().getProbabilityEstimate() - 0.5*myAlgoMonteCarlo.getResult().getConfidenceLength(0.95), ", ", myAlgoMonteCarlo.getResult().getProbabilityEstimate() + 0.5*myAlgoMonteCarlo.getResult().getConfidenceLength(0.95), "]" )
print( "" )
alpha = 0.90 
convergenceGraphMonteCarlo = myAlgoMonteCarlo.drawProbabilityConvergence(alpha) 
#convergenceGraphMonteCarlo.draw("convergenceGrapheMonteCarlo", 640, 480) 


#convergence_graph_monte_carlo = algo_monte_carlo.drawProbabilityConvergence(alpha)
View(convergenceGraphMonteCarlo)

### Reliability analysis using analytical model and MCS

In [ ]:
X_random_vector = ot.RandomVector(ot.Distribution(myDistribution))
G = ot.CompositeRandomVector(model, X_random_vector)

threshold = 30
myEvent = ot.ThresholdEvent(G, ot.ComparisonOperator(ot.Greater()), threshold)
myEvent.setName("Deviation > 30 cm")
#
maximumOuterSampling = 40000
blockSize = 100
coefficientOfVariation = 0.10
#
experiment = ot.MonteCarloExperiment()
myAlgoMonteCarlo = ot.ProbabilitySimulationAlgorithm(myEvent, experiment)
myAlgoMonteCarlo.setMaximumOuterSampling(maximumOuterSampling)
myAlgoMonteCarlo.setBlockSize(blockSize)
myAlgoMonteCarlo.setMaximumCoefficientOfVariation(coefficientOfVariation)
myAlgoMonteCarlo.setConvergenceStrategy(ot.HistoryStrategy(ot.Full()))
#
myAlgoMonteCarlo.run()

print( "Number of evaluations of the limit state function = ", myAlgoMonteCarlo.getResult().getOuterSampling()* myAlgoMonteCarlo.getResult().getBlockSize() )
print( "Monte Carlo probability estimation = ", myAlgoMonteCarlo.getResult().getProbabilityEstimate() )
print( "Variance of the Monte Carlo probability estimator = ", myAlgoMonteCarlo.getResult().getVarianceEstimate() )
print( "0.95 Confidence Interval = [", myAlgoMonteCarlo.getResult().getProbabilityEstimate() - 0.5*myAlgoMonteCarlo.getResult().getConfidenceLength(0.95), ", ", myAlgoMonteCarlo.getResult().getProbabilityEstimate() + 0.5*myAlgoMonteCarlo.getResult().getConfidenceLength(0.95), "]" )
print( "") 
alpha = 0.90 
convergenceGraphMonteCarlo = myAlgoMonteCarlo.drawProbabilityConvergence(alpha) 
#convergenceGraphMonteCarlo.draw("convergenceGrapheMonteCarlo", 640, 480) 


#convergence_graph_monte_carlo = algo_monte_carlo.drawProbabilityConvergence(alpha)
View(convergenceGraphMonteCarlo).show()